In [1]:

from fastapi import HTTPException
from typing import Any, Dict, Optional
import json
import hmac
import hashlib
import httpx

AGENT_SECRET = 'dEcartes2026'
IP = '34.55.196.22'

def json_dumps(payload: dict) -> bytes:
    return json.dumps(payload, separators=(",", ":"), sort_keys=True).encode()


def call_agent_hmac(ip, path, secret=AGENT_SECRET, json_body=None, method="POST", timeout=5):
    url = f"http://{ip}:8000{path}"
    
    if method == "DELETE":
        # Use path param (fqdn) as body for HMAC
        fqdn = path.split("/")[-1]
        body_bytes = fqdn.encode()
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers = {
            "Content-Type": "text/plain",
            "X-Signature": signature,
        }
        res = httpx.request("DELETE", url, content=body_bytes, headers=headers, timeout=timeout)

    else:
        headers = {"Content-Type": "application/json"}
        body_bytes = json_dumps(json_body) if json_body else b"{}"
        signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
        headers["X-Signature"] = signature
        
        if method == "POST":
            res = httpx.post(url, content=body_bytes, headers=headers, timeout=timeout)
        elif method == "GET":
            body_bytes = b""
            signature = hmac.new(secret.encode(), body_bytes, hashlib.sha256).hexdigest()
            headers = {"X-Signature": signature}
            res = httpx.get(url, headers=headers, timeout=timeout)

    try:
        res.raise_for_status()
    except httpx.HTTPStatusError as e:
        raise HTTPException(
            status_code=e.response.status_code,
            detail=f"Agent error: {e.response.text}"
        )
    return res


In [9]:
# Load JSON from file
with open("../json/dnsproof.org.json", "r") as f:
    payload = json.load(f)

# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/push",
    json_body=payload
)

print(response.json())

{'status': 'success'}


#### DNSSEC enabling

In [10]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/sign/dnsproof.org"
)

print(response.json())

{'rrsig_records': ['SOA 8 2 3600 20260210060635 20260113060635 14428 @ rcFWuucHm9f0lMjH/5yD7hnTrYYOnlFz fL2DvUdS1YWoBiHv0VUaxd3Jm9ANBNQJ qDu9hnJv57hGxoAilTEGqNAC9jO+Bzuw HZ1+2/5qCZsEiG74V6Bl8/kueyVKjbl5 IMP0hGIsa+zX8TS90wOzsZl5ud/YplTV F7BQtgaYtCM=', 'NS 8 2 3600 20260210060635 20260113060635 14428 @ edq5bIUm7dXOm2s7ynCRSLY33Xf9xDGj OIjW5ArJGWlM+gFgp9pSYzEJcZWbZopA BabhHkNAylCWEwbcjHco6KzqcWso8qAc f8P0Lfty4LexEvGPPQHuHMiZRX18GcDr Gy/myJuWNCy6X0ksxtTTBALsMj8egxFa KiD11s8nyo0=', 'MX 8 2 3600 20260210060635 20260113060635 14428 @ rvePauCgRF1/mX+zhVQh6hk1SBF3U47A 0iJRIE0s+NzrBWEVBQinFPXUnfmxn5Ol DbOQkjTllGaPQE8or7esqlOd4m/2I9/1 08dK3DGAfkURKa0x92AninH767Co9gDu +IbVFWypH29AGpScAWth/4g2rjfvYqXU 1mdxD5EP8ww=', 'TXT 8 2 3600 20260210060635 20260113060635 14428 @ uwEoSEoc+JdU52gInUhe3WhZAabv/EFC YBUZ+uaS803di6F44KW8wo3zm8Vfzdza Xj2+u95IvkmJ16FUD75h31IOnov8RmxP IktO53GGyKD84fKtwq6YwA7dq85UBy8u hpExY67dy8tR9TZE8Eks+hXevss+9z5e 8AKkuoRnkF8=', 'DNSKEY 8 2 3600 20260210060635 20260113060635 58421 @ 

#### DNSSEC disabling

In [54]:
# Send request
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/disable/dnsproof.org"
)

print(response.json())

{'status': 'success'}


#### DNSSEC status

In [8]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/status/dnsproof.org",
    method="GET"
)

print(response.json())

{'ksk_created_at': '2026-01-13T03:50:27.392334', 'zsk_created_at': '2026-01-13T03:50:27.372334', 'ds_digest': 'EE8584003AE1BFB7353309661702F844801C54914E7E315744AA9B9EAB8EE47C', 'key_tag': 32616, 'algorithm': 8, 'days_since_last_key_creation': 0}


#### DNSSEC key rotation

In [69]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/rotate/dnsproof.org"
)

print(response.json())

{'rrsig_records': ['SOA 8 2 3600 20260209090924 20260112090924 14349 @ X8CEngj59Dh9aqEwm+eo6immLKUw8Czx pcFEisSOt2EziH50CH5yNUbYR/5DJ6Pg 5DUhW55pr0ADnUZn89MqroDJbRvZJ6Q6 eShK0clUFwHWwo0CTUjQsh+/BacpUMwl 752H4vd+Vycn2wUgjOKVVgk/i6J3q3tr uu9UeJ6ealY=', 'NS 8 2 3600 20260209090924 20260112090924 14349 @ pfmFk4tuoDhyLbMGV4TDyiBh39RO5dPK jsSpa5EUVNUuBD6ULyVLfxUJ3T9VBWWW J3GZlQMc2V027xSA0DZBBPwcl2KgiekN ta5uoovbiU2vma4dihp8NPgJGF/10l4f IqViI1JM6wKf/hi6qpVUerlD2e2VPr7g 4KTcg3DzN88=', 'MX 8 2 3600 20260209090924 20260112090924 14349 @ jfEEIWG2vS+t/g397kVxD3QmuX7HjUyk DP4GsXTPkzrCTfnVyypavYWKcF/4A/FZ WQvu+5AQk0DyzRc08+gHi8BaesYd85pV p8syka/+Vhl3R25WT9FXD/JVr6P4u5LJ g+FUp8V/K830T2gFtvOBhWih6ACH/uS6 Yd/6BCV1X1E=', 'TXT 8 2 3600 20260209090924 20260112090924 14349 @ de6xatrK6sitbHTdDFeU0V76hp89H81o nk8YjLsGEn/9DXTU5QDjVG257yZk+PMk LRwF1PmclEACKEGqLY16lxnRPxVNL7mz jPy0xnBQmP36le5ftTB3Z2I7NhEm2MBK sBaoCECmEeG+M+AwirJYKe8BWiTba1zj Xi8CHiMAa98=', 'DNSKEY 8 2 3600 20260209090924 20260112090924 6706 @ B

#### DNSSEC ZSK rotation

In [72]:
response = call_agent_hmac(
    ip=IP,
    path="/internal/dns/dnssec/rotate/zsk/dnsproof.org"
)

print(response.json())

{'rrsig_records': ['SOA 8 2 3600 20260209110119 20260112110119 56955 @ Yhv2OMqRtNIYlL+JyqQAutoxpa1yG8JE fnk/jHyNxIuIVQ6tIr2hjpamoph1B2bp de+x5GcpEQWQOeJLoJVvJ53FwYkT+7k9 tEANGV9+LA6NDFSlkRlGtC/l1b9LHjpI ZLcYdsCZDxHR7/MwCU7Ulze3+kZw13Ck +WeQk40s4Ic=', 'NS 8 2 3600 20260209110119 20260112110119 56955 @ BA8U1l81IRp5H8r9nBQFjj/4tcrrLaF0 qfkrT8xOZZPjEwKURddue11U6MnQ+Byp t4eW9yT+LviMjpmZpGO8ZBQSeam6ebGu W2sG1EAXnWO+vbC+n9b3g/mgLW0VCSzj Fl0OkrJk0EwvJ4p0/htKnaC2+QBOxcEP 7CXQFUWDTas=', 'MX 8 2 3600 20260209110119 20260112110119 56955 @ qqBD2QTd5nr7+0OguvYMeMUIJ0H/vSUj aZlqeIgcDQZcz/uX+gPw27pEtNJFvnn/ 9JmcWIi/C/KXIRlYmx5dP+Cclh8/tqOt wnxLIJDhy7qCjX/dn2drNcp3p4o5wiBA A22yguvnQpjKQp2d2oQemTyHnbePllGR vQW6SCpvXFo=', 'TXT 8 2 3600 20260209110119 20260112110119 56955 @ ExOy9JnYaMHxxDSrUO+PDLelN1SstmXR Hw3vZkSt5+n1gww2qfxrxFbwa7q0tvjn EsfZEPNBUa2FOjvfPy65XxLAq0w/nCaA 7I3jbAzogrgMLXBkhtX+BqWd2MKuP7Dh s0z72TFrIkXk1rTJs13mLF8uXjuu0KAX jM50HVBsyto=', 'DNSKEY 8 2 3600 20260209110119 20260112110119 6706 @ G